<a href="https://colab.research.google.com/github/thomaslu678/gee-test/blob/main/clean/2_ee_task_lst_all_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTE: Requires points.csv (list of points within digitized buffer zone)

In [38]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [39]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [40]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gee-481701')

## Test the API

Test the API by printing the elevation of Mount Everest.

In [41]:

# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


# Define Areas of Interest

In [42]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import scipy.stats as stats
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
import geopandas as gpd
from shapely.geometry import Point
import requests

In [43]:
landsat_collections = {
    "L4": {
        "collection": "LANDSAT/LT04/C02/T1_L2",
        "sr": ["SR_B1", "SR_B3", "SR_B4", "SR_B5", "SR_B7"],
        "lst": "ST_B6"
    },
    "L5": {
        "collection": "LANDSAT/LT05/C02/T1_L2",
        "sr": ["SR_B1", "SR_B3", "SR_B4", "SR_B5", "SR_B7"],
        "lst": "ST_B6"
    },
    "L7": {
        "collection": "LANDSAT/LE07/C02/T1_L2",
        "sr": ["SR_B1", "SR_B3", "SR_B4", "SR_B5", "SR_B7"],
        "lst": "ST_B6"
    },
    "L8": {
        "collection": "LANDSAT/LC08/C02/T1_L2",
        "sr": ["SR_B2", "SR_B4", "SR_B5", "SR_B6", "SR_B7"],
        "lst": "ST_B10"
    },
    "L9": {
        "collection": "LANDSAT/LC09/C02/T1_L2",
        "sr": ["SR_B2", "SR_B4", "SR_B5", "SR_B6", "SR_B7"],
        "lst": "ST_B10"
    }
}

# Functions

In [44]:
def mask_qa_pixel(img: ee.Image):
    # QA_PIXEL is the name of the fetched file that provides info
    # about pixel quality assurance
    qa = img.select("QA_PIXEL")

    # Fetch bits (0/1) describing if an environmental condition
    # is present
    cloud        = qa.bitwiseAnd(1 << 3).neq(0)
    cloud_shadow = qa.bitwiseAnd(1 << 4).neq(0)
    snow         = qa.bitwiseAnd(1 << 5).neq(0)
    cirrus       = qa.bitwiseAnd(1 << 2).neq(0)

    mask = (
        # Or condition for all bad conditions
        cloud
        .Or(cloud_shadow)
        .Or(snow)
        .Or(cirrus)

        # Not flips the conditions so any remaining pixels
        # have none of the conditions, and are "good" quality.
        .Not()
    )

    # Apply the mask to the input image
    return img.updateMask(mask)

In [45]:
def scale_sr(img, sr_bands):
    scaled = (
        img.select(sr_bands)
        .multiply(2.75e-05)
        .add(-0.2)
        .rename(["Blue", "Red", "NIR", "SWIR1", "SWIR2"])
    )
    return img.addBands(scaled)

def scale_lst(img, lst_band):
    lst = (
        img.select(lst_band)
        .multiply(0.00341802)
        .add(149.0)
        .rename("LST_K")
    )
    return img.addBands(lst)


# Loop over all points

In [46]:
input_df = pd.read_csv('/content/sample_data/points.csv') # id column is called point_id

In [47]:
def df_to_points_fc(df):
    feats = []
    for _, r in df.iterrows():
        geom = ee.Geometry.Point(float(r["long"]), float(r["lat"]))
        feats.append(
            ee.Feature(geom, {
                "point_id": r["point_id"]
            })
        )
    return ee.FeatureCollection(feats)

points_fc = df_to_points_fc(input_df)

In [48]:
city_geom = points_fc.geometry().convexHull(maxError=1)

In [49]:
def sample_landsat_collection(cfg, mission, city_geom, points_fc):

    col = (
        ee.ImageCollection(cfg["collection"])
        .filterBounds(city_geom)
        .map(mask_qa_pixel)
        .map(lambda img: scale_sr(img, cfg["sr"]))
        .map(lambda img: scale_lst(img, cfg["lst"]))
    )

    bands_to_sample = ["Blue", "Red", "NIR", "SWIR1", "SWIR2", "LST_K"]

    def per_image(img):
        samples = img.select(bands_to_sample).sampleRegions(
            collection=points_fc,
            geometries=False,
            scale=None
        )

        return samples.map(lambda f: f.set({
            "mission": mission,
            "time": ee.Date(img.get("system:time_start"))
                    .format("YYYY-MM-dd HH:mm:ss")
        }))

    return col.map(per_image).flatten()

In [50]:
collections = []

for mission, cfg in landsat_collections.items():
    fc = sample_landsat_collection(cfg, mission, city_geom, points_fc)
    collections.append(fc)

final_fc = ee.FeatureCollection(collections).flatten()

In [51]:
ee.batch.Export.table.toDrive(
    collection=final_fc,
    description="landsat_city_points_minimal",
    fileFormat="CSV",
    selectors=[
        "point_id",
        "mission",
        "time",
        "Blue",
        "Red",
        "NIR",
        "SWIR1",
        "SWIR2",
        "LST_K"
    ]
).start()

Link to GEE Tasks:

https://code.earthengine.google.com/tasks

In [54]:
ee.data.getTaskStatus("44VVXRT2OWWMAM7NMRBGII4S")

[{'state': 'RUNNING',
  'description': 'landsat_city_points_minimal',
  'priority': 100,
  'creation_timestamp_ms': 1770277138852,
  'update_timestamp_ms': 1770277181786,
  'start_timestamp_ms': 1770277151377,
  'task_type': 'EXPORT_FEATURES',
  'attempt': 1,
  'batch_eecu_usage_seconds': 9.100181579,
  'id': '44VVXRT2OWWMAM7NMRBGII4S',
  'name': 'projects/gee-481701/operations/44VVXRT2OWWMAM7NMRBGII4S'}]

In [53]:
 ee.data.listOperations()


[{'name': 'projects/gee-481701/operations/44VVXRT2OWWMAM7NMRBGII4S',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'PENDING',
   'description': 'landsat_city_points_minimal',
   'priority': 100,
   'createTime': '2026-02-05T07:38:58.852900Z',
   'updateTime': '2026-02-05T07:38:58.852900Z',
   'startTime': '1970-01-01T00:00:00Z',
   'type': 'EXPORT_FEATURES'}},
 {'name': 'projects/gee-481701/operations/EVG3QH5XUKDUQTXWIY6R7OPA',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'CANCELLING',
   'description': 'landsat_lst_city_points',
   'priority': 100,
   'createTime': '2026-02-05T07:37:04.216919Z',
   'updateTime': '2026-02-05T07:37:48.242447Z',
   'startTime': '2026-02-05T07:37:08.020214Z',
   'type': 'EXPORT_FEATURES',
   'attempt': 1,
   'batchEecuUsageSeconds': 17.881847381}},
 {'name': 'projects/gee-481701/operations/SPYCHORQANELZNXZJWY4QDXP',
  'metadata': {'@type': 